In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
import os
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
import dask_image.imread
import numpy as np
from skimage import feature, morphology, img_as_float
import scipy.ndimage as ndi
from random import random
import sklearn
import cc3d
import pandas as pd
import seaborn as sns
import math

In [ ]:
def get_vec(label, idx):
    pca = PCA()
    xl, yl, zl = np.where(label==idx)
    voxelsl = [v for v in zip(list(xl.astype(np.float32)), list(yl.astype(np.float32)), list(zl.astype(np.float32)))]
    pca.fit(voxelsl)
    return np.array([int(np.mean(xl)), int(np.mean(yl)), int(np.mean(zl))]), pca.components_[2]*100, pca.explained_variance_[2]

In [ ]:
def get_vec_mito(label, idx):
    pca = PCA()
    xl, yl, zl = np.where(label==idx)
    voxelsl = [v for v in zip(list(xl.astype(np.float32)), list(yl.astype(np.float32)), list(zl.astype(np.float32)))]
    pca.fit(voxelsl)
    return np.array([int(np.mean(xl)), int(np.mean(yl)), int(np.mean(zl))]), pca.components_[0]*100

In [ ]:
def angle(x,y):
    dot_xy = np.dot(x,y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    cos = dot_xy / (norm_x*norm_y)
    rad = np.arccos(cos)
    theta = rad * 180 /np.pi
    return theta

In [ ]:
def compute_angle(x,y):
    return(min(angle(x,y), angle(-x, y)))

In [ ]:
def compute_all_mito(df, df_la, labels_mito, labels_la):
    npu_mito, count_mito = np.unique(labels, return_counts=True)
    angles_m = [] #angle mean
    mito_nums=[] #mito number
    mito_V_pxs=[] #mito volume
    mito_V_pxs_mito=[]
    lamellar_no=[] #lamellar id 
    lamellar_v_pxs=[] #lamellar volume
    angles=[] # lamellar mito angle
    covs = [] # pc3 cov
    cov_vs = [] #pc3 cov/lamellar volume
    mito_num=0
    for i in range(1, len(npu_mito)):
        if count_mito[i]>1000:
            _, mito_vec = get_vec_mito(labels, npu_mito[i])
            cristaes = np.where(labels==npu_mito[i], labels_la, 0)
            npu, count = np.unique(cristaes, return_counts=True)   
            angles_temp = []
            mito_num=mito_num+1
            mito_V_pxs_mito.append(count_mito[i])
            if len(npu)==1:
                angles_m.append(None)
            else:
                for idx in range(1,len(npu)):
                    if count[idx]>1000:
                        _, vec, cov = get_vec(cristaes, npu[idx])
                        ang = compute_angle(vec, mito_vec)
                        angles_temp.append(ang)
                        mito_nums.append(mito_num)
                        mito_V_pxs.append(count_mito[i])
                        lamellar_v_pxs.append(count[idx])
                        angles.append(ang)
                        covs.append(cov)
                        cov_vs.append(cov/count[idx])
                print(angles_temp)
                angles_m.append(np.mean(angles_temp))
                lamellar_no = lamellar_no + list(range(1,len(angles_temp)+1))
    if len(df)==0:
        crop_num=1
    else:
        crop_num=int(list(df['crop_No'])[-1])+1

    df_temp = pd.DataFrame({'crop_No': [crop_num]*(len(angles_m)),
                            'mito_No' : list(range(1, mito_num+1)),
                 'mito_V_px' : mito_V_pxs_mito,
                'angles' : angles_m})
    print(len([crop_num]*(len(mito_nums))))
    print(len(mito_nums))
    print(len(mito_V_pxs))
    print(len(lamellar_no))
    print(len(angles))
    print(len(covs))
    print(len(cov_vs))
    df_la_temp= pd.DataFrame({'crop_No' :[crop_num]*(len(mito_nums)),
                         'mito_No' :mito_nums,
                         'mito_V_px' :mito_V_pxs,
                         'lamellar_No' :lamellar_no,
                         'lamellar_V_px' : lamellar_v_pxs,
                         'angles' :angles,
                         'lamellar_PC3cov' :covs,
                         'cov/V': cov_vs})
    return pd.concat([df, df_temp]).reset_index(drop=True), pd.concat([df_la, df_la_temp]).reset_index(drop=True)

In [ ]:
# calculate control
df= pd.DataFrame({'crop_No' :[],
                  'mito_No' :[],
                 'mito_V_px' : [],
                'angles' : []})
df_la= pd.DataFrame({'crop_No' :[],
                     'mito_No' :[],
                     'mito_V_px' : [],
                     'lamellar_No' :[],
                     'lamellar_V_px' : [],
                     'angles' : [],
                     'lamellar_PC3cov' :[],
                     'cov/V':[]})
for idx in range(5):
    print(idx)
    filename_pattern = os.path.join(f'./dataset/annotations_shCtrl/cropped_00{idx+1}/devided_mito', '*png')
    mito = dask_image.imread.imread(filename_pattern)
    mito_np = np.zeros(mito.shape, np.uint8)
    for i in range(len(mito)):
        mito_np[i] = (mito[i].compute()>0)*255
    connectivity = 6 
    labels = cc3d.connected_components(mito_np, connectivity = connectivity)
    filename_pattern_l = os.path.join(f'./dataset/annotations_shCtrl/cropped_00{idx+1}/lamellar_with_postprocessing', '*png')
    la = dask_image.imread.imread(filename_pattern_l)
    la_np = np.zeros(la.shape, np.uint8)
    for i in range(len(la)):
        la_np[i] = (la[i].compute()>0)*255
    diamond = ndi.generate_binary_structure(rank=3, connectivity=1)
    eroded = ndi.binary_erosion(la_np, diamond, iterations=1)
    labels_la = cc3d.connected_components(eroded, connectivity = connectivity)
    df, df_la = compute_all_mito(df, df_la, labels, labels_la)

In [ ]:
# calculate shOPA1
df2= pd.DataFrame({'crop_No' :[],
                  'mito_No' :[],
                 'mito_V_px' : [],
                'angles' : []})
df2_la= pd.DataFrame({'crop_No' :[],
                     'mito_No' :[],
                     'mito_V_px' : [],
                     'lamellar_No' :[],
                     'lamellar_V_px' : [],
                     'angles' : [],
                     'lamellar_PC3cov' :[],
                     'cov/V':[]})
for idx in range(5):
    filename_pattern = os.path.join(f'./dataset/annotations_shOpa1/cropped_00{idx+1}/devided_mito', '*png')
    mito = dask_image.imread.imread(filename_pattern)
    mito_np = np.zeros(mito.shape, np.uint8)
    for i in range(len(mito)):
        mito_np[i] = (mito[i].compute()>0)*255
    connectivity = 6 
    labels = cc3d.connected_components(mito_np, connectivity = connectivity)
    filename_pattern_l = os.path.join(f'./dataset/annotations_shOpa1/cropped_00{idx+1}/lamellar_with_postprocessing', '*png')
    la = dask_image.imread.imread(filename_pattern_l)
    la_np = np.zeros(la.shape, np.uint8)
    for i in range(len(la)):
        la_np[i] = (la[i].compute()>0)*255
    diamond = ndi.generate_binary_structure(rank=3, connectivity=1)
    eroded = ndi.binary_erosion(la_np, diamond, iterations=1)
    labels_la = cc3d.connected_components(eroded, connectivity = connectivity)
    df2, df2_la = compute_all_mito(df2, df2_la, labels, labels_la)

In [ ]:
df2_la['gene'] = ['shOPA1']*len(df2_la)
df_la['gene'] = ['shCtrl']*len(df_la)


In [ ]:
df2['gene'] = ['shOPA1']*len(df2)
df['gene'] = ['shCtrl']*len(df)

In [ ]:
df.to_csv('angles_ctrl.csv', index=False)
df2.to_csv('angles_opa1.csv', index=False)
df_la.to_csv('lame_ctrl.csv', index=False)
df2_la.to_csv('lame_opa1.csv', index=False)

In [ ]:
df = pd.read_csv('angles_ctrl.csv')
df2 = pd.read_csv('angles_opa1.csv')
df_la = pd.read_csv('lame_ctrl.csv')
df2_la = pd.read_csv('lame_opa1.csv')

In [ ]:
df_c = pd.concat([df,df2])
df_c

In [ ]:
df_la_c = pd.concat([df_la,df2_la])
df_la_c

In [ ]:
df_temp = df_la_c[df_la_c['gene']=='shCtrl']
xs = []
ys = []
for i in range(len(df_temp)):
    theta = math.radians(df_temp['angles'][i])
    x = math.cos(theta)/df_temp['lamellar_PC3cov'][i]
    y = math.sin(theta)/df_temp['lamellar_PC3cov'][i]
    xs.append(x)
    ys.append(y)
df_temp = df_la_c[df_la_c['gene']=='shOPA1']
xs2 = []
ys2 = []
for i in range(len(df_temp)):
    theta = math.radians(df_temp['angles'][i])
    x = math.cos(theta)/df_temp['lamellar_PC3cov'][i]
    y = math.sin(theta)/df_temp['lamellar_PC3cov'][i]
    xs2.append(x)
    ys2.append(y)

In [ ]:
plt.quiver([0]*len(xs), [0]*len(xs), xs, ys, angles='xy', scale_units='xy', scale=1, color='green', alpha=0.1)
plt.quiver([0]*len(xs2), [0]*len(xs2), xs2, ys2, angles='xy', scale_units='xy', scale=1, color='magenta', alpha=0.1)
plt.quiver(sum(xs)/len(xs), sum(ys)/len(ys), angles='xy', scale_units='xy', scale=0.25, color='green')
plt.quiver(sum(xs2)/len(xs2), sum(ys2)/len(ys2), angles='xy', scale_units='xy', scale=0.25, color='magenta')
plt.axis('square')
plt.xlim(0,2.25)
plt.ylim(0,2.25)
plt.xticks( np.arange(0, 2.5, 0.5))
plt.yticks( np.arange(0, 2.5, 0.5))
plt.savefig("quiver.svg", format="svg")